In [1]:
import pandas as pd
from os import path, walk
import re

In [2]:
source_path = '../data/water/processed/'

In [20]:
def enumerate_files(folder, pattern):
    for dirpath, dirnames, filenames in os.walk(folder):
        for f in filenames:
            if re.match(pattern, f):
                yield path.join(dirpath, f)

def read_and_rank(path):
    df = pd.read_csv(path, low_memory=False)
    df['Date'] = df['StartDate'] + ' ' + df['StartTime']
    df['Rank'] = df.sort_values('Date')\
                    .groupby(['LocationIdentifier', 'Pollutant'])\
                    .cumcount() + 1
    return df.drop('Date', 1)

def merge_station_files(folder):
    dfs = [pd.read_csv(f, low_memory=False) 
           for f in enumerate_files(folder, r'\w\w_station-.*\.csv')]
    return pd.concat(dfs)

def merge_rank_result_files(folder):
    dfs = [read_and_rank(f) 
           for f in enumerate_files(folder, r'\w\w_result-.*\.csv')]
    return pd.concat(dfs)

In [18]:
print("Concatenating stations")
merge_station_files(source_path)\
    .to_csv(path.join(source_path, 'all-station.csv'))

Concatenating stations


In [19]:
print("Concatenating results")
merge_rank_result_files(source_path)\
    .to_csv(path.join(source_path, 'all-result.csv'))


Concatenating results


In [ ]:
fips_code_path = '../data/fips_codes.csv'

In [ ]:
fips = pd.read_csv(fips_code_path)
fips.columns

In [ ]:
state_index = fips[['State', 'StateId']].drop_duplicates()

In [ ]:
state_to_id = state_index.set_index('State').to_dict()['StateId']
id_to_state = state_index.set_index('StateId').to_dict()['State']

In [ ]:
def get_county_data(state):
    county = fips.ix[fips.State == state]
    return county[['County', 'CountyId']].drop_duplicates()

def get_county_to_id(state):
    county_index = get_county_data(state)
    return county_index.set_index('County').to_dict()['CountyId']

def get_id_to_county(state):
    county_index = get_county_data(state)
    return county_index.set_index('CountyId').to_dict()['County']

In [ ]:
get_county_to_id('AK')

In [ ]:
dir, file = path.split('/test/foo.txt')
file[:2]